In [35]:
from pathlib import Path

from itables import show
import pandas as pd

In [52]:
!ls ./output

connectedcomponents-epinions1  pagerank-epinions1  trianglecounting-epinions1
connectedcomponents-facebook   pagerank-facebook   trianglecounting-facebook
connectedcomponents-google     pagerank-google	   trianglecounting-google


In [37]:
def parse_metrics(metrics_file: Path, num_cpus: int, amount_ram: str) -> pd.DataFrame:
    data = {}
    with open(metrics_file, 'r') as file:
        section = None
        for line in file:
            line = line.strip()
            if not line:
                continue
            if line.startswith('[') and line.endswith(']'):
                section = line[1:-1]
                continue
    
            key, value = line.split('=', 1)
            key = key.strip()
            value = value.strip()
    
            if section:
                data[key[1:]] = value

    algorithm, dataset = metrics_file.name.split('-')
    data['num_cpus'] = num_cpus
    data['amount_ram'] = amount_ram
    data['algorithm'] = algorithm
    data['dataset'] = dataset
    return pd.DataFrame([data])

In [53]:
output_dir = Path("./output")
metrics = []
for metric_file in output_dir.iterdir():
    df = parse_metrics(metric_file, 6, "3G")
    metrics.append(df)

graph_metrics = pd.concat(metrics)
show(graph_metrics)

In [54]:
graph_metrics.to_parquet("./final_metrics/graphchi_metrics_6cpu_3G.parquet")